## Cookbook for using VectaraDB with Embedchain

### Step-1: Install embedchain package

In [1]:
#!pip install embedchain[vectara]

!python -m pip install -e ~/dev/embedchain

import os
os.environ['VECTARA_CUSTOMER_ID'] = '1169579801'
os.environ['VECTARA_OAUTH_CLIENT_ID'] = '7vleevf47e4s86a3qln6k95h97'
os.environ['VECTARA_OAUTH_SECRET'] = '285svsv942iel08unn1uf3quhlnbpq6aj30j6nhs383nmlpq33e'

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


Obtaining file:///Users/ofer/dev/embedchain
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for embedchain (pyproject.toml) ... done
  Created wheel for embedchain: filename=embedchain-0.1.59-py3-none-any.whl size=9339 sha256=ea5cdd374f78714b2c3dd1fe404655a2986caade71c9e2715c48b8a9c51ac775
  Stored in directory: /private/var/folders/48/8_815vxj0vl2z5_28gys249r0000gn/T/pip-ephem-wheel-cache-mi8uh2fn/wheels/c1/10/22/d1bfbf20a6f1021e5a5c8f177f263d6d6ba4ea62066fbf8671
Successfully built embedchain
  Attempting uninstall: embedchain
    Found existing installation: embedchain 0.1.59
    Uninstalling embedchain-0.1.59:
      Successfully uninstalled embedchain-0.1.59


### Step-2: Set environment variables needed for Vectara

If you do not have a Vectara account yet, first [sign up](https://vectara.com/integrations/embedchain) for your account.
You can find the env variables on [Vectara console](https://console.vectara.com/console/apiAccess/appClients) and on your [OpenAI dashboard](https://platform.openai.com/account/api-keys). 

In [2]:
import os
from embedchain import App

#os.environ["OPENAI_API_KEY"] = "sk-xxx"
#os.environ["VECTARA_CUSTOMER_ID"] = "xxx"
#os.environ["VECTARA_OAUTH_CLIENT_ID"] = "xxx"
#os.environ["VECTARA_OAUTH_SECRET"] = "xxx"

### Step-3 Create embedchain app and define your config

Since Vectara provides a GenAI platform (we call it RAG-in-a-box) you do not need an embedding provider as this is done inside the platform.
Furthermore, Vectara does its own chunking internally, so we configure the chunking to be a no-op.

In [3]:
app = App.from_config(
    config={
        "vectordb": {
            "provider": "vectara",
            "config": {
                "collection_name": "vectara-demo"
            }
        },
        "chunker": {
            "chunk_size": 1000000,
            "chunk_overlap": 0
        }
    })

DEBUG response=<Response [200]>


### Step-4: Add data sources to your app

For our example we will add the full documentation website of Vectara. 
By choosing "docs_site" as the data type, we ask EmbedChain to crawl and ingest all the pages in the Vectara documentation:

In [4]:
app.add("https://docs.vectara.com/", data_type="docs_site")

DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
Adding 40 new documents to Vectara (out of 40 submitted to indexing)...
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>


'd1f20d25579de0cf550ce26446eaeaaa'

### Step-5: All set. Now start asking questions related to your data

In [5]:
questions = [
    "What is Vectara?",
    "How do I index documents in Vectara?",
    "what is hybrid search and how does it work in Vectara?",
    "What is RAG?",
    "How does MMR work?"
]

data = []
for q in questions:
    answer = app.query(q)
    data.append([q, answer])

df = pd.DataFrame(data=data, columns=["question", "answer"])

DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>
DEBUG response=<Response [200]>


In [6]:
df.style.set_properties(**{'text-align': 'left'})

,question,answer
0,What is Vectara?,Vectara is a platform that offers generative AI search capabilities and is designed for developers with an API-first approach. It provides easy ingestion of data and simple APIs for integration into applications. Vectara can be used as an answer engine and offers superior language understanding for ingestion and retrieval of information.
1,How do I index documents in Vectara?,"To index documents in Vectara, you can use either the Standard Indexing API or the File Upload API. For the Standard Indexing method, you can use the REST endpoint provided by Vectara at the following URL: https://api.vectara.io/v1/index. You will need to include the necessary headers, such as the customer_id, in your REST calls to interact with the Index service. Alternatively, you can use the File Upload method, which exposes an HTTP endpoint for uploading and indexing files into a corpus. Both methods allow you to make your data available for search and retrieval in Vectara."
2,what is hybrid search and how does it work in Vectara?,"Hybrid search in Vectara is a text retrieval approach that combines traditional, keyword-based search with semantic search using neural models. It offers a powerful and flexible way to retrieve relevant information from text. In Vectara, hybrid search works by blending partial, exact, and Boolean text matching with neural models. This allows for a more comprehensive understanding of user queries and enables Vectara to provide contextually accurate responses. With hybrid search, you can include exact keyword matches even if they were absent from Vectara's training data, such as product SKUs. You also have the option to disable neural retrieval entirely and use exact term matching. Additionally, you can incorporate typical keyword modifiers like a NOT function, exact phrase matching, and wildcard prefixes of terms. To enable hybrid search in Vectara, you need to specify a value called ""lambda"" at query time, specifically under the corpusKey. The lambda value can range from 0 to 1, with 0 representing traditional keyword-based search and 1 representing neural retrieval."
3,What is RAG?,"RAG stands for Retrieval Augmented Generation. It is a technology used in generative AI systems to ensure that the generated content is both verifiable and anchored to the supplied data. RAG minimizes the occurrence of hallucinations, which are inaccurate or misleading information commonly found in generative AI systems. It summarizes search results that answer complex queries directly and provides citations to ground these search results in facts from the data."
4,How does MMR work?,I don't have enough information to answer the query about how MMR works.


In [7]:
app.query("What is Vectara?", citations=True)

DEBUG response=<Response [200]>


('Vectara is a platform that offers generative AI search capabilities and is designed for developers with an API-first approach. It provides easy ingestion of data and simple APIs for integration into applications. Vectara can be used as an answer engine and offers superior language understanding for ingestion and retrieval of information.',
 [("Why certain birds are allowed for the most random reason? We have part of a\nunique employee handbook ready for you to upload into a Vectara corpus,\nand we'll guide you through the ingestion and question answering process step by step. Step 1. Create a Vectara account \u200b To get started with Vectara , go to https://console.vectara.com/signup or\nclick Get Started Free at vectara.com. After you make an account, you can\ncreate your first corpus and upload your first document! Step 2. Create your first corpus \u200b Before you can ask the data about bringing your velociraptor to the\noffice, you first need to create a corpus.",
   {'lang': 'e